In [3]:
from sqlalchemy import create_engine, inspect, text, Engine
import pandas as pd
import numpy as np

In [4]:
engine = create_engine(
    "postgresql+psycopg2:///country_club?host=/var/run/postgresql", 
    # echo=True,
    # isolation_level="AUTOCOMMIT"
)
table_names = inspect(engine).get_table_names()

In [5]:
inspect(engine).get_columns("bookings")

[{'name': 'bookid',
  'type': SMALLINT(),
  'nullable': False,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'facid',
  'type': SMALLINT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'memid',
  'type': SMALLINT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'starttime',
  'type': TIMESTAMP(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'slots',
  'type': SMALLINT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None}]

In [6]:
def get_results_frame(query, engine: Engine, **query_vars):
    with engine.connect() as conn:
        rs = conn.execute(text(query), query_vars)
        results = pd.DataFrame(rs.fetchall(), columns=rs.keys())
    return results

for table_name in table_names:
    test_query = f"SELECT * FROM {table_name} LIMIT 1;"
    print(table_name, get_results_frame(test_query, engine).T)

bookings                              0
bookid                       0
facid                        3
memid                        1
starttime  2012-07-03 11:00:00
slots                        2
members                                  0
memid                            0
surname                      GUEST
firstname                    GUEST
address                      GUEST
zipcode                          0
telephone           (000) 000-0000
recommendedby                 None
joindate       2012-07-01 00:00:00
facilities                                  0
facid                            0
name                Tennis Court 1
membercost                     5.0
guestcost                     25.0
initialoutlay                10000
monthlymaintenance             200
expense_label                 None


# Syntax, Types, and Data Transforming Functions

## General points

In general, SQL queries are whitespace indifferent and end in a `;`. Commands and table names are case-insensitive, though by convention the former are typed in all caps.

Comments are indicated with `--` or `/*` ... `*/`. The latter can encompass multiple lines.

When there is no ambiguity, columns can be referred to by name alone. If there are multiple tables references in a query with shared columns names, they can be distinguished with `tbl.col`.

Selected values and queried tables can be aliased with `AS`. The alias can then be used in elements of the query that are processed later (see below).

Values can be cast to different types with the `::` operator or `CAST(expression AS type)` (note that the latter is the SQL standard syntax). This will attempt to parse values from strings, e.g. date or time.


## Table metadata

Information about the database is stored in the `INFORMATION_SCHEMA` table. For example, to see every data table in the database:

```sql
SELECT table_name
FROM INFORMATION_SCHEMA.TABLES
WHERE table_schema = 'public';
```


To see the names and data types for a given table:

```sql
SELECT
    column_name,
    data_type
FROM INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'tbl';
```

In [7]:
table_query = """
    SELECT 
        table_catalog,
        table_name 
    FROM INFORMATION_SCHEMA.TABLES
    WHERE table_schema = 'public';
"""

get_results_frame(table_query, engine)

,table_catalog,table_name
0,country_club,bookings
1,country_club,members
2,country_club,facilities


## Numeric types

See the other notebook for details on the different numeric types.

To round values for display, using `ROUND(value, places)`. The number of `places` defaults to 0, i.e. whole numbers. Negative values will round to the nearest 10, 100, etc.

The function `TRUNC(value, places)` has the same syntax.

Sequences of numbers can be generated using `GENERATE_SERIES(start, end, step)`. This produces a column with the indicated parameters (inclusive of `end`). Two of of these can be used to create bins to show a distribution.

## Strings

### Literals

String expressions are enclosed in single quotes, and column or table names are enclosed in double quote when necessary (e.g. when they contain spaces). For example:
```sql
SELECT * FROM "this is a table" WHERE "this is a column" = 'a string';
```

### String manipulation

In some dialects, including SQLite and PostgreSQL, strings are concatenated with `||`, while in others like MySQL, this is done with the `CONCAT()` function:
```sql
'this' || ' and that'
CONCAT('this', ' and that')
```
Note that `||` can be strung together consecutively, and `CONCAT()` can take any number of parameters. Non-strings will be converted automatically.

Other string functions include:
- The case of a string can be manipulated with `UPPER(str)`, `LOWER(str)`, and `INITCAP(str)` (i.e. title case).
- `REVERSE(str)`
- `CHAR_LENGTH(str)` or `LENGTH(str)`
- `REPLACE(stringval, 'pattern to match', 'replacement')`.
- `POSITION('pat' IN strval)` or `STRPOS(strval, 'pat')`: returns the first 1-indexed starting position of a pattern, or 0 if not found.
- `LEFT(strval, n)` `RIGHT(strval, n)`: returns the first or last `n` characters of a string.
- `SUBSTR[ING](strval, start, length)` or `SUBSTRING(strval FROM start FOR length)`. Note that lengths that extend beyond end of string will be ignored, so a lenght of `LENGTH(strval)` will always ensure getting the rest of the string. These functions will accept starting positions less than 1 but will eat up length until a valid index is reached. That is to say `SUBSTR('anything', 0, 1)` is always an empty string.
- `TRIM()`: removes characters from ends of string. Given just a string value, it will remove whitespace from beginning and end. `LTRIM` and `RTRIM` can only trim from one side each.
- Padding: `LPAD` and `RPAD`. Both take three parameters: the string value, the desired length, and the string to append as padding (defaulting to `' '`). Note that this function truncates strings longer than the indicated length and will truncate the padding string if it does not fit evenly into the needed padding.
- `SPLIT_PART(strval, delimiter, part])`; negative values of `part` count from end.

### Matching

The `=` operator performs a character-for-character comparison. It is also possible to pattern match with the `LIKE` or `ILIKE` operators. In pattern strings, `%` represents a free wildcard, so `LIKE 'a%'` will match `'a'`, `'aa'`, `'aba'`, etc. The `_` wildcard matches exactly one character. In PostgreSQL, `LIKE` is case sensitive, while `ILIKE` is not. In SQLite, `LIKE` is case insensitive and the latter keyword does not exist.

PostgreSQL also has support for "full text" or natural language matching, which allows for variations in capitalization, spelling, and evens word forms (singular and plural, etc). This is implemented through two distinct data types: `tsvector`s to represent documents and `tsquery`s to represent the patterns to be matched. Strings must be converted to these with either `to_tsvector('text')` and `to_tsquery('pattern')` or casting. The expression `document @@ pattern` tests whether a `tsvector` contains a `tsquery`.

A query string is composed of words linked by operators:
- `&` indicates both elements should appear somewhere in the document, not necessarily together
- `|` indicates that at least one element should match
- `!` indicates that the following element should **not** match
- `<->` indicates that the two elements should appear adjacent and in that order. This is equivalent to `<1>`; larger values of the number indicate larger gaps between the words.

Note that for the first three, doubled version of the operator can be used to link two `tsquery`s, while `<->` can also be used as an operator on its own.

The functions `plainto_tsquery()` and `phraseto_tsquery()` convert multi-word strings to queries with different connections between the world: `&` for the former and `<n>` for the latter (after dropping stopwords).

## Date and time types

Date and time and be compared using standard operators, where future dates and times are greater than earlier.

Two timestamp values can be subtracted to get an interval, or an interval can be added to a value. Date differences are expressed as an integer representing days. Interval literals can be expressed using `INTERVAL 'n unit'`, where unit can be any standard component of the type. Date-time subtraction can also be accomplished with `AGE(dt1, dt2)`, which returns the interval from the second to the first argument. In the absence of a second argument, the function returns the interval to the present.

The components of date and time types can be accessed with `EXTRACT(field FROM dtval)` or `DATE_PART('field', dtval)`. Valid `field`s  include `YEAR`, `QUARTER`, `MONTH`, `DAY`, `DOW` (for numerical day of the week, Sunday is 0), `HOUR`, etc.

The granularity of date-time values can be reduced using `DATE_TRUNC('field', dtval)`. Note that this preserves the type, replacing truncated fields with their minimum value. So, for example `DATE_TRUNC('month', NOW())` returns a timestamp of the first day of the current month with time `00:00:00`.

The current day or time can be accessed via:
- `NOW()` or `CURRENT_TIMESTAMP`: a millisecond `timestamp` with timezone. The timezone can be removed by casting `:: timestamp`. `CURRENT_TIMESTAMP(n)` specifies that the time value should be rounded to `n` decimal places after the second value.
- `CURRENT_DATE` returns a `date`
- `CURRENT_TIME` returns a `time` with time zone.

Date-Time values can be converted into formatted strings using `to_char(dtval, 'format')`. For example, `'Day, Month dd yyyy'` will yield `Monday, January 01 2001`. Note that `'d'` is treated as `'dow'`. To escape field characters, using double quotes: e.g. to get `'Q1'`. use `'"Q"q'`.

To create calendar columns (useful for comparing periods including those with no observations): `generate_series(from, to, interval)`. As usual in SQL, the series includes the endpoint if it falls on the interval. NOTE: when generating date points, it is better to anchor in the start of the period (especially month), because, e.g. the first of month plus `'1 month'` is always the first of the next month. So, to get the last of the month, make a series of first days and subtract 1.

Note that when joining a calendar table to data table, empty dates will be `NULL`; `coalesce(val, 0)` can be used to instead output 0 for those rows. See the example below:

In [8]:
date_series_query = """
WITH months AS
    (SELECT generate_series(date_trunc('year', min(starttime)), date_trunc('month', max(starttime)), INTERVAL '1 month') as month
    -- NOTE: using 'year' means that there will be empty months up until the beginning of the data
    FROM bookings
    ORDER BY month)
SELECT 
    to_char(months.month, 'Month yyyy') as "Month",
    coalesce(num_bookings, 0) as "Number of bookings"
FROM (SELECT 
        date_trunc('month', starttime) as month,
        count(*) as num_bookings
    FROM bookings
    GROUP by month -- grouping here reduces the needed matches for the join
    ) as b
    RIGHT JOIN months -- preserves every month
    ON b.month = months.month
"""

get_results_frame(date_series_query, engine)

,Month,Number of bookings
0,January 2012,0
1,February 2012,0
2,March 2012,0
3,April 2012,0
4,May 2012,0
5,June 2012,0
6,July 2012,658
7,August 2012,1472
8,September 2012,1913


## Arrays

Arrays of any other data type are represented by adding square brackets to the end of type name (including arrays). So the type of a 2d array of ints is `int[][]`. Array literals can be represented as strings with curly brackets, e.g. `'{{1 ,2}, {3, 4}}'`. Strings within an array need to be marked with **double** quotes. An alternative is to use `ARRAY[val1, val2...]`. In this syntax, strings should be enclosed in single quotes as usual.

Arrays are indexed using square brackets, *starting with 1*. So the value 1 in the array literal in the previous paragraph would be `arr[1][1]`.

The `ANY` or `ALL` operators work wtih arrays: the preceding test is performed on each element. So `1 = ANY arr` would be true.

There are also array-specific operators:
- The operators `<@` and `@>` test whether the array *pointed towards* is a subset of the other. So, `'{1}' <@ arr` is true, but `'{1}' @> arr` would be false.
- The `&&` operator tests for any common elements.
- Concatenation: the `||` operator can either append an element to an array or combine two arrays.




# The SELECT query

## Elements and their order of operations

A `SELECT` statement has the following basic format:

```sql
SELECT
    [Columns and expressions separated by commas]
FROM
    [Table expressions and joins]
[Options]
```

The expression in a select statement or filter can be either column names or functions. These include basic arithmetic (`+`, `-`, `*`, `/` NB if both values are integers the latter will be integer division).

Options include:
- Filtering rows: `WHERE [condition]`. The condition can be any expression calculable on a single row.
- Grouping results: `GROUP BY [columns or columns, separated by commas]`. To filter on properties of groups, following this `GROUP BY` expression with `HAVING [condition]`. The condition must be an expression with a single value for the group, e.g. the grouping column value or an aggregate function of other columns.
- Ordering results: `ORDER BY [column or columns, separated by commas]`. By default, sorting is ascending; to sort in descending order, add `DESC` after the column name.
- Limiting results: `LIMIT [n]`.

The order of processing is:
1. Sources: the `FROM` clause, including subqueries and joins
1. Filters: the `WHERE` clause
1. The `GROUP BY` clause and its filters
1. Selection: querying column values or evaluating functions, as well as aliases. Column names cannot be referred to in the `FROM` or `WHERE` clauses but can be in the `GROUP BY` clause. Also, columns can be referred to be 1-indexed number.
1. Set operations (see below)
1. The `ORDER BY` clause
1. The `LIMIT` clause

## Filtering results

As noted above, filtering can be accomplished in two ways:
1. `WHERE` clauses, which filter on a row-by-row basis
1. `GROUP BY ... HAVING` clauses, which filter on values evaluated on groups

Certain filters can be accomplished using one or the other, for example choosing only certain groups: filtering by row on the values of the grouping column will yield the same result as filtering the groups on the same value.

Conditional expression include:
- Boolean operators: `AND`, `OR`, `NOT`. The last can be combined with many of the keywords below: `NOT IN`, `NOT LIKE`, etc.
- Standard mathematical comparison operators: `=`, `<>`, `<`, `<=`, `>`, `>=`. Note that the second 
- Group comparison: these have syntax `[column expression] [operator(s)] ()` where the `()` contains a set, which can be either a literal list, e.g. `(1, 2, 3)`, an array data type, or a `SELECT` subquery.
    - `IN` or `<@`: returns true if the value is found in the list or subquery results (`IN`) or array (`<@`).
    - `[comparison operator] ANY ()`: returns true if the operator returns true for the expression and any item in the list. So, (I think) `= ANY (SELECT...)` is equivalent to `IN`. See the examples in the next cell.
    - `[comparison operator] ALL ()`: returns true if the operator returns true for the expression and all items in the list. See the example in the cell after next for one use case: finding rows with the maximum values in a column.
    - `EXISTS (SELECT ...)` is a test for whether the subquery has any rows at all. The inverse is `NOT EXISTS`.
- String comparison: the comparison operators work for strict equivalence or alphabetical order. See above on pattern and natural language matching.
- Ranges of values: `BETWEEN [lower] AND [upper]`. This range includes the lower and upper bounds.
- Missing values can be captured or excluded with `IS NULL` or `IS NOT NULL`.

In [9]:
query_any = """
SELECT
    name
FROM
    facilities
WHERE facid = ANY (SELECT facid FROM facilities WHERE monthlymaintenance > 200);
"""

query_any2 = """
SELECT
    name
FROM
    facilities
WHERE monthlymaintenance > 200;
"""

query_in = """
SELECT
    name
FROM
    facilities
WHERE facid IN (SELECT facid FROM facilities WHERE monthlymaintenance > 200);
"""

print(np.all(get_results_frame(query_any, engine) == get_results_frame(query_any2, engine)))
print(np.all(get_results_frame(query_any2, engine) == get_results_frame(query_in, engine)))

True
True


In [10]:
# This query returns the names of records with the maximum value in the monthlymaintenance column
query_all = """
SELECT
    name,
    monthlymaintenance
FROM facilities
WHERE monthlymaintenance >= ALL (SELECT monthlymaintenance FROM facilities);
"""

get_results_frame(query_all, engine)

,name,monthlymaintenance
0,Massage Room 1,3000
1,Massage Room 2,3000


In [29]:
# This query illustrates that `EXISTS` excludes rows where it finds an empty query result
# Here, every query is empty, so nothing is returned

query_exists = """
SELECT *
FROM facilities
WHERE EXISTS (SELECT LIMIT 0);
"""

get_results_frame(query_exists, engine)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance,expense_label


## Conditional values

Values from an expression can be replaced conditionally using a `MATCH` statement:
```sql
SELECT
    CASE 
        WHEN condition1 THEN result1
        WHEN condition2 THEN result2
        ...
        ELSE default_result
    END as some_name
    ...
```

Note that evaluation of arms is greedy: once one arm is matched, that is the value returned. In the absence of an `ELSE` clause, any row for which no arm is matched will be marked `NULL`. Note that this behavior can be leveraged for one common use of `MATCH` statements: to provide counts, sums, or averages that include only certain rows, without excluding those rows from other calculations.

In addition, the type of all results must be compatible: either the same or able to be coerced to the same type.

The specific task of filling in `NULL` calues can be handled with the `COALESCE(val1, val2...)` function. This function takes a list of values (column expressions or literals) and returns the first non-null.

## Combining column values into strings

If the output of a query is just the order of a sequence of values (say ranked names), it can be cleaner to display them horizontally as a comma-separated list. This can achieved with the function `STRING_AGG(vals, separator)`
```sql
SELECT
    STRING_AGG(name, ', ')
FROM (
    SELECT
        name as ordered_val
    FROM tbl
    ORDER BY val
) as ordered_val_tbl;
```

In [11]:
fac_order_query = """
    SELECT
        name,
        monthlymaintenance
    FROM
        Facilities
    ORDER BY monthlymaintenance;
"""

string_agg_query = """
    SELECT
        STRING_AGG(ordered_name, ', ') as "Facilities in ascending order by monthly maintenance cost"
    FROM (
        SELECT
            name as ordered_name
        FROM
            Facilities
        ORDER BY monthlymaintenance
    ) as ordered_fac;
"""

print(get_results_frame(fac_order_query, engine))
print(get_results_frame(string_agg_query, engine))

              name  monthlymaintenance
0     Table Tennis                  10
1    Snooker Table                  15
2       Pool Table                  15
3  Badminton Court                  50
4     Squash Court                  80
5   Tennis Court 1                 200
6   Tennis Court 2                 200
7   Massage Room 2                3000
8   Massage Room 1                3000
  Facilities in ascending order by monthly maintenance cost
0  Table Tennis, Snooker Table, Pool Table, Badmi...       


## Saving query results

If a particular query is going to be referred to frequently, it can be efficient to save it.

Within a single query, a subquery can be defined in advance with a **common table expression (CTE)**.
```sql
WITH cte_name AS SELECT ...
SELECT ... FROM name ...;
```
Multiple CTE definitions can be joined by commas.

Queries can also be saved for later reference as **views** or **temporary tables**:
```sql
CREATE VIEW view_name AS SELECT ...;
```
The view can subsequently be referred to like a normal table. The syntax is the same for temporary tables, where `VIEW` is replaced with `TEMP TABLE`

There is also this alternative syntax, for creating temporary or permanent tables:
```sql
SELECT ...
    INTO TEMP TABLE
    FROM ...
```
Note that the destination is placed after the columns selected and before the `FROM` clause.

Temporary tables behave like regular tables (e.g. values can be added, deleted, and modified after creation, without affecting original), but they exist only for the length of the current client session. They do not require full editing priveleges to create. 

## Exporting query results to csv

Data can be exported to files using the `COPY` command or `\copy` meta-command. The syntax is the same, but the former is strictly speaking a server command, while the latter is executed by the client. In some cases, users only have permission for former.

The `...` below represents what data is to be exported. This can be a table name, a table name with selected columns indicated in parantheses, or a query.
```sql
COPY ... 
    TO 'file path' 
    WITH 
        FORMAT csv, 
        HEADER true;
```

# Combining Table and Queries

## JOIN statements

A join extends one table horizontally by matching records on some key. This can be one-to-one, which basically just linked records. One-to-many or many-to-one link details about some entity that appears multiple times in a table. A `JOIN` statement identifies the type of join and the key with which to link records:
```sql
FROM tbl1
    JOIN tbl2
    ON tbl1.col1 = tab2.col2
```
To match rows based on the values of multiple columns, the equivalences with `AND`. Strictly, this can be any logical expression, including transformations of columns or inequalities. For example, to match a value into a ranged bin defined by two columns on the right table, `...ON val >= lower AND val < upper`. (Note that `BETWEEN` is not suitable here, because it is inclusive of both ends.) If the key has the same name in both tables, the `ON` clause can be replaced with `USING (key)`.

The types of joins define how matches between the tables are handled:
- `INNER JOIN`, also the default, returns only records for which matching keys are found in both tables
- `LEFT JOIN` or `RIGHT JOIN` returns all records in the first or second table (respectively). For records without matches in the other table, the columns from the other table will be `NULL`.
- `FULL JOIN` includes all rows from both tables.
- `CROSS JOIN` is a distinct type, which instead of using a key, simply returns every possible combination of rows from the two tables (i.e., with $n$ rows in one table and $m$ rows in the other, the joined table will have $n \times m$ rows).

Repeated `JOIN` statements are evaluated sequentially. It can be helpful to alias tables to make references to columns more concise.

## Self Joins

A special case of joins is when one column in a table links the record to other rows in the same table. A self join unrolls the linked information. This uses the same syntax as any other join, except that the same table name is referred to twice. However, in this case, aliasing is required in order to refer to the two different roles of the same table.

## Set operations to concatenate query results

Tables can also be combined vertically if the number and type of columns match.
```sql
SELECT ...
UNION
SELECT ...
[display options]
```
Each component `SELECT` has its own filtering options. `ORDER BY` and `LIMIT` at the end apply to the combined results. Column names are determined the *first* `SELECT` statement.

The set operators are:
- `UNION`: combine all rows, once for each distinct record. `UNION ALL` includes duplicates.
- `INTERSECT`: include only matching rows.
- `EXCEPT`: includes only rows that appear in eactly one of the queries.

## Subqueries

An additional way to combine values in different tables is through `SELECT` statements inside other clauses of a query.

For example, filtering rows in one table based on values in another table can be accomplished with an inner join:
```sql
SELECT
    tbl1.col
  FROM tbl1
    INNER JOIN tbl2
    ON tbl1.key = tbl2.id;
```

Or by a subquery filter:
```sql
SELECT
    tbl1.col
  FROM tbl1
 WHERE tbl1.key in (
    SELECT id FROM tbl2
);
```

This is a **simple** nested query because the `SELECT` stetement within the `WHERE` clause needs to be evaluated only once. A **correlated** query is one that depends in some way on the rows of the main query and thus must be evaluated separately for each result. For example, to select rows based on the number of appearances in a second one-to-many table:

```sql
SELECT tbl1.col
  FROM tbl1
 WHERE 10 <
    (SELECT
        count(*)
       FROM tbl2
      WHERE tbl2.id = tbl1.key);
```

Often correlated subqueries will be less efficient because they require a search be done for each row separately, whereas a join creates a temporary table once. I suspect that most efficient way to do what the previous code block does would be a left join followed by grouping (or perhaps grouping before the join). I would assume that the joining and grouping algorithms are well-optimized compared to this kind of row-by-row matching.

# Aggregation

## Kinds of aggregation

SQL includes various functions that serve to produce aggregate values for groups of records. These can be used as columns for a summary query or with window functions to combine row-level and aggregate values.

### Central 

- `AVG(col)`
- `MODE(col)`
- See percentils for median

### Totals
- `SUM(col)`
- `COUNT([DISTINCT] col)`. Use `*` to give a count of records.

### Association
- `CORR(x, y)`
- `COVAR_POP()`, `COVAR_SAMP()`

### Distribution

In addition to `MIN(col)` and `MAX(col)`, there is a function to find arbitrary percentiles:
```sql
PERCENTILE_CONT(0.1) WITHIN GROUP (ORDER BY col) AS tenth_percentile
```
This returns the value within `col` corresponding to the specified percentile, interpolating if necessary. `PERCENTILE_DISC` returns the specific value in the data that is the first at or beyond the specified percentile. If used as a summary function, replace `OVER` with `WITHIN GROUP`.

For statistical measures of distribution, there are paired sample and population versions:
- `VAR_POP()`, `VAR_SAMP()`, and `VARIANCE()` as an alians for the latter.
- `STDDEV_POP()`, `STDDEV_SAMP()`, and `STDDEV()` as an alians for the latter.


## Summary functions for all results or for groups

Summary functions included in a simple `SELECT` statement will return the summary for all records or by groups if these are defined by a `GROUP BY`.

Note that non-summary-function elements can be included in the same `SELECT` clause only if these are the bases for group formation. Note that in the query below, there is a one-for-one relationship between `memid` and name, but the engine still requires they all be included as groups. (It is possible that keys are no properly configured here.)

In [12]:
group_query1 = """
    SELECT 
        recommendedby,
        COUNT(*) as "Members recommended"
    FROM members
    GROUP BY recommendedby
    ORDER BY "Members recommended" DESC;
"""

group_query2 = """
    SELECT 
        recommender.firstname || ' ' || recommender.surname as "Name",
        COUNT(*) as "Members recommended"
    FROM members as recommended
        JOIN members as recommender
        ON recommended.recommendedby = recommender.memid
    GROUP BY recommended.recommendedby, recommender.firstname, recommender.surname
    HAVING recommended.recommendedby IS NOT NULL
    ORDER BY "Members recommended" DESC;
"""

get_results_frame(group_query2, engine)

,Name,Members recommended
0,Darren Smith,5
1,Tracy Smith,3
2,Ponder Stibbons,2
3,Janice Joplette,2
4,Jemima Farrell,2
5,Florence Bader,1
6,Millicent Purview,1
7,Matthew Genting,1
8,Tim Rownam,1
9,Gerald Butters,1


### Group and grand totals

While window functions can be used to include totals within rows, adding separate rows for totals is different. This is done with additional keywords on the `GROUP BY` clause: `ROLLUP()` or `CUBE()`.

The following query will show the sum for each year and category, as well as totals for each year, and a grand total at the bottom:
```sql
SELECT
    year,
    category,
    SUM(val)
FROM tbl
GROUP BY ROLLUP (year, category);
```

If `category` and `year` are reversed, the result will instead by totals of each category for all years. To only output the yearly totals but omit the grand total, the last line would be:
```sql
GROUP BY year, ROLLUP (category)
```

To include **both** totals of each categories for all years and totals for each year of all categories, use `CUBE`:
```sql
SELECT
    year,
    category,
    SUM(val)
FROM tbl
GROUP BY CUBE (year, category);
```

Note that for all of these, the column that is being aggregated will be displayed as `null`. To replace this will a label, use `COALESCE(col, 'label')`, which in this context returns the value of `col` if non-null, otherwise `'label'`.

It is possible to flexibly specify the levels of aggregation with the `GROUPING SETS ()` keyword. The parantheses contain the grouping criteria to use, each contained by paranthesis and separated by a comma. A single grouping category will result in the reporting subtotals for such groups. The set `()` indicates grand totals. So, the equivalent to the first query above is:
```sql
SELECT
    year,
    category,
    SUM(val)
FROM tbl
GROUP BY GROUPING SETS ((year, category), (year), ());
```

This can be thought of as the union of queries each grouping by one of the elements in the `GROUPING SETS` list.

In [43]:
# This is equivalent to Cube.
# Note that if month is used in the grouping clause, the table will output Null instead of total

grouping_query = """
SELECT
    coalesce(date_part('month', starttime)::text, 'total') as month,
    coalesce(facid::text, 'total') as Facility,
    count(*)
FROM bookings
GROUP BY GROUPING SETS ((date_part('month', starttime), facid),(date_part('month', starttime)), (facid), ())
ORDER BY 1, 2;
"""

get_results_frame(grouping_query, engine)

,month,facility,count
0,7,0,88
1,7,1,68
2,7,2,56
3,7,3,51
4,7,4,123
5,7,5,12
6,7,6,75
7,7,7,75
8,7,8,110
9,7,total,658


## Window functions

Window functions using the `OVER ()` keyword allow rows to output values that depend on other rows in the table. Some of these operations can be performed using subqueries, but generally speaking window functions are more efficient and less verbose.

The parantheses following `OVER` contain keywords indicating the relevant window, if required.

### Functions based on order

#### Row numbers

A relatively simple example involves numbering results.

Arbitrary row numbers based on the order in which records are returned (i.e. before any *subsequent* `ORDER BY` clauses) can be generated with 
```sql 
ROW_NUMBER() OVER () AS row_number
```
To alter the order of rows before numbers are assigned, `ORDER BY` needs to be within the `ORDER` clause:
```sql 
ROW_NUMBER() OVER (ORDER BY col) AS row_number
```
As with `ORDER BY` in general, this gives ascending order; use `DESC` to rank from largest to smallest.

#### Ranking

The `RANK` function is similar to `ROW_NUMBER` except that rows with equivelent values are given the same rank, with the rank incremented accordingly for the next distinct value.
```sql
RANK() OVER (ORDER BY col) AS col_rank
```

The function `DENSE_RANK()` has the same behavior for ties, but does not increment. In other words, in descending order, a dense rank of $n$ represents "the $n^\text{th}$ highest distinct value."

#### Comparing values across rows

The `LAG(col, n)` function returns the value in a column `col`, offset by `n` rows upwards (i.e. "previously"). This can be used to capture changing values:
```SQL
SELECT
    tdate,
    val <> LAG(val, 1) OVER (ORDER BY tdate) AS changed
FROM
    tbl;
```

The directions inverse of `LAG` is `LEAD`.

The beginning and end of a window can be fetched with `FIRST_VALUE(col)` or `LAST_VALUE(col)`. For the latter, note that the window by default ranges from the beginning of the series to the current row. To get the last value in the entire series:
```sql
LAST_VALUE(col) OVER (ORDER BY col RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS final_value
```

#### N-tiles

In addition to returning the percentile value of a sequence, it is possible to identify which "~tile" a given value is in with `NTILE(n)`. In this function `n` is the number of chunks in which to break the data, so `n=100` is equivalent to percentiles.
```sql
NTILE(10) OVER (ORDER BY col) as decile
```
In this example, a value of 1 means that row is the bottom tenth, while a value of 10 means it is in the top tenth.

This can also be thought of as breaking results into "pages" of roughly equal size.

Ranking example:

In [13]:
rank_query1 = """
    SELECT 
        membercost, 
        RANK() OVER(ORDER BY membercost DESC) as member_cost_rank
    FROM facilities;
"""

get_results_frame(rank_query1, engine)

,membercost,member_cost_rank
0,9.9,1
1,9.9,1
2,5.0,3
3,5.0,3
4,3.5,5
5,0.0,6
6,0.0,6
7,0.0,6
8,0.0,6


Lag example:

In [14]:
lag_query = """
    SELECT 
        day,
        COUNT(*) as number_of_bookings,
        COUNT(*) > LAG(COUNT(*), 1) OVER (ORDER BY day) as increased
    FROM ( SELECT
            *,
            EXTRACT(MONTH FROM starttime ) AS month,
            EXTRACT(DAY FROM starttime ) AS day 
        FROM bookings
    ) as daily
    WHERE month = 8
    GROUP BY day;
"""

get_results_frame(lag_query, engine)

,day,number_of_bookings,increased
0,1,29,None
1,2,43,True
2,3,39,False
3,4,37,False
4,5,33,False
5,6,42,True
6,7,35,False
7,8,49,True
8,9,36,False
9,10,51,True


### Grouping

Within window functions, groups are defined with `PARTITION BY`. For example, to output both the value of a row and the group and global averages (note the use of `OVER()`):
```sql
SELECT
    col,
    group,
    AVG(col) OVER () as global_average,
    AVG(col) OVER (PARTITION BY group) as group_average
FROM
    tbl;
```

In [15]:
group_avg_query = """
    SELECT
        name as "Facility Name",
        slots,
        ROUND(AVG(slots) OVER(), 1) as "Global Average Slots",
        ROUND(AVG(slots) OVER(PARTITION BY Bookings.facid), 1) as "Facility Average Slots"
    FROM
        Bookings
        JOIN Facilities USING(facid)
    ORDER BY starttime
    LIMIT 15;
"""

get_results_frame(group_avg_query, engine)

,Facility Name,slots,Global Average Slots,Facility Average Slots
0,Massage Room 1,2,2.3,2.2
1,Pool Table,1,2.3,1.1
2,Table Tennis,2,2.3,2.1
3,Pool Table,1,2.3,1.1
4,Squash Court,2,2.3,2.5
5,Snooker Table,2,2.3,2.0
6,Tennis Court 1,3,2.3,3.2
7,Pool Table,1,2.3,1.1
8,Squash Court,2,2.3,2.5
9,Massage Room 1,2,2.3,2.2


### Sliding windows

In addition to calculating on groups, window functions can be calculated on moving windows within the data. E.g., a 5-step, centered moving average can be calculated as:
```sql
SELECT
    AVG(col) OVER (ROWS BETWEEN 2 PRECEDING AND 2 FOLLOWING) AS rolling_average
FROM
    tbl
ORDER BY seq;
```

The order can also between defined with `OVER (ORDER BY ...)`. Other range keywords include:
- `CURRENT ROW`
- `UNBOUNDED` in place of a number value to express every observation `PRECEDING` or `FOLLOWING`.
- `GROUPS` instead of `ROWS` counts by "peer groups", or unique values in the current ordering, i.e. the same logic used by `RANK()`. `RANGE` counts by values of the ordering column (which must be singular). Note that for date or time types, interval strings (e.g. "5 days") can be used.

Note that whereas the window for `OVER ()` is the entire group, with `OVER (ORDER BY ...)` the default window is `RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW`.

In [16]:
# This query calculates the cumulative sum of slots booked within each month, for one member and one facility.
# Note that the two formulations (one with ORDER BY and one with ROWS) are equivalent

rolling_query_1 = """
SELECT
    starttime,
    slots,
    SUM(slots) OVER (PARTITION BY EXTRACT(MONTH FROM starttime) ORDER BY starttime) as "Cumulative total for the month"
FROM
    Bookings
WHERE memid = 1 AND facid = 1
ORDER BY starttime;
"""

rolling_query_2 = """
SELECT
    starttime,
    slots,
    SUM(slots) OVER (PARTITION BY EXTRACT(MONTH FROM starttime) ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as "Cumulative total for the month"
FROM
    Bookings
WHERE memid = 1 AND facid = 1
ORDER BY starttime;
"""

print(np.all(get_results_frame(rolling_query_1, engine)==get_results_frame(rolling_query_2, engine)))
get_results_frame(rolling_query_2, engine)

True


,starttime,slots,Cumulative total for the month
0,2012-07-08 15:00:00,3,3
1,2012-07-08 17:30:00,3,6
2,2012-07-09 19:00:00,3,9
3,2012-07-12 11:30:00,3,12
4,2012-07-13 15:00:00,3,15
5,2012-07-16 08:00:00,3,18
6,2012-07-17 10:00:00,3,21
7,2012-07-19 11:30:00,3,24
8,2012-07-21 09:30:00,3,27
9,2012-07-23 10:00:00,3,30


# Pivot tables

PostgreSQL offers pivot table functionality as an extension through the crosstab function.

First, the extension must be enabled with
```sql
CREATE EXTENSION IF NOT EXISTS tablefunc;
```
Note that creating extension requires permissions beyond creating and editing tables.

To create a pivot table showing values on a measure by `category` for each of two values in a `pivot_col`:
```sql
SELECT * FROM CROSSTAB($$
    SELECT
        category :: CATTYPE
        pivot_col
        measure :: VALTYPE
    FROM tbl
    GROUP BY category, pivot_col
    ORDER BY category, pivot_col
$$) AS ct (
    category CATTYPE,
    pivot_val1 VALTYPE,
    pivot_val2 VALTYPE
);
```

Note that the way this works, values are assigned to the pivoted columns based on position. Values are read down the final column of the source table and filled left to right, row by row in the output table. Thus, the output column names do not need to reflect the values of the pivoted column.

The types must match exactly, and the values in the internal select statement should be cast accordingly to be certain.

In [17]:
query_pivot = """
SELECT * FROM CROSSTAB($$
    SELECT
        facid,
        Month,
        SUM(Slots) :: INT
    FROM (SELECT
            facid,
            EXTRACT(Month FROM starttime) as Month,
            Slots
        FROM Bookings
    ) AS month_slots
    GROUP BY facid, Month
    ORDER BY facid, Month
$$) AS ct (
    facility SMALLINT,
    July INT,
    August INT,
    September INT
);
"""

get_results_frame(query_pivot, engine)

,facility,july,august,september
0,0,270,459,591
1,1,207,483,588
2,2,180,459,570
3,3,104,304,422
4,4,264,492,648
5,5,24,82,122
6,6,164,400,540
7,7,156,326,426
8,8,117,322,471
